# Data Cleaning
In this notebook, we will be cleaning the data sets that were reviewed in the Data Collection notebook. For modeling we need to combine the features we want to use into on dataframe for modeling.
<br><br> This includes:
 - Combining MovieLens and IMDb reviews into one dataframe
 - 
 - 

In [1]:
# Imports
import pandas as pd

## Combining MovieLens and IMDb Reviews
### Reformatting Movie IDs
Our data from MovieLens comes with three different IDs for each movie, but we will be merging our data using the IMDb ID from our IMDb review data. The MovieLens IDs only have a number for IMDb ID, whereas the other data sets have it formatted as an 9-character code starting with "tt". Toy Story, for example, is "tt0114709" in the IMDB data and only "114709" in the MovieLens Data.

In [2]:
ml_links = pd.read_csv('../Data/Large-Data/MovieLens-25M/links.csv')
ml_links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [3]:
ml_links.sort_values(by = 'imdbId', ascending=True)

,movieId,imdbId,tmdbId
46249,172063,1,16612.0
32269,140539,3,88013.0
50293,180695,7,159895.0
16842,88674,8,105158.0
24188,120869,10,774.0
...,...,...,...
61985,207453,11057912,636593.0
62071,207714,11101550,640427.0
62387,209051,11108064,642749.0
62312,208711,11168100,642203.0


In [4]:
# Lets add as a new column called 'imdb_reformatted'
ml_links['imdb_reformatted'] = 'tt' + ml_links['imdbId'].astype(str).str.zfill(7)

ml_links.sort_values(by = 'imdbId', ascending=True)

,movieId,imdbId,tmdbId,imdb_reformatted
46249,172063,1,16612.0,tt0000001
32269,140539,3,88013.0,tt0000003
50293,180695,7,159895.0,tt0000007
16842,88674,8,105158.0,tt0000008
24188,120869,10,774.0,tt0000010
...,...,...,...,...
61985,207453,11057912,636593.0,tt11057912
62071,207714,11101550,640427.0,tt11101550
62387,209051,11108064,642749.0,tt11108064
62312,208711,11168100,642203.0,tt11168100


### Reformatting MovieLens Ratings
Our MovieLens ratings are scored out of 5, with 10 possible scores. The IMDb scores also have 10 options, but are formatted as 1-10. We can scale these to match so we can combine them into one set of ratings.

In [5]:
ml_ratings = pd.read_csv('../Data/Large-Data/MovieLens-25M/ratings.csv')
ml_ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [6]:
# Multiply the MovieLens rating by 2 so we have the same 1-10 rating scale as IMDb
ml_ratings['scaled_rating'] = ml_ratings['rating'] * 2
ml_ratings.head()

,userId,movieId,rating,timestamp,scaled_rating
0,1,296,5.0,1147880044,10.0
1,1,306,3.5,1147868817,7.0
2,1,307,5.0,1147868828,10.0
3,1,665,5.0,1147878820,10.0
4,1,899,3.5,1147868510,7.0


In [7]:
# We need to add our new IMDb ID column to our MovieLens ratings
ml_ratings_for_merge = ml_ratings[['userId', 'movieId', 'scaled_rating']]
ml_ratings_for_merge

,userId,movieId,scaled_rating
0,1,296,10.0
1,1,306,7.0
2,1,307,10.0
3,1,665,10.0
4,1,899,7.0
...,...,...,...
25000090,162541,50872,9.0
25000091,162541,55768,5.0
25000092,162541,56176,4.0
25000093,162541,58559,8.0


In [8]:
ml_ratings_for_merge = pd.merge(ml_ratings_for_merge, ml_links, on='movieId', how='left')
ml_ratings_for_merge

,userId,movieId,scaled_rating,imdbId,tmdbId,imdb_reformatted
0,1,296,10.0,110912,680.0,tt0110912
1,1,306,7.0,111495,110.0,tt0111495
2,1,307,10.0,108394,108.0,tt0108394
3,1,665,10.0,114787,11902.0,tt0114787
4,1,899,7.0,45152,872.0,tt0045152
...,...,...,...,...,...,...
25000090,162541,50872,9.0,382932,2062.0,tt0382932
25000091,162541,55768,5.0,389790,5559.0,tt0389790
25000092,162541,56176,4.0,952640,6477.0,tt0952640
25000093,162541,58559,8.0,468569,155.0,tt0468569


In [9]:
# We need to rename our columns to match the MovieLens reviews
ml_ratings_for_merge = ml_ratings_for_merge.rename({'userId': 'user_id', 'imdb_reformatted': 'imdb_id'}, axis='columns')
ml_ratings_for_merge.head()

,user_id,movieId,scaled_rating,imdbId,tmdbId,imdb_id
0,1,296,10.0,110912,680.0,tt0110912
1,1,306,7.0,111495,110.0,tt0111495
2,1,307,10.0,108394,108.0,tt0108394
3,1,665,10.0,114787,11902.0,tt0114787
4,1,899,7.0,45152,872.0,tt0045152


In [10]:
# Take only the columns we will use for modeling
ml_ratings_for_merge = ml_ratings_for_merge[['user_id', 'imdb_id', 'scaled_rating']]
ml_ratings_for_merge

,user_id,imdb_id,scaled_rating
0,1,tt0110912,10.0
1,1,tt0111495,7.0
2,1,tt0108394,10.0
3,1,tt0114787,10.0
4,1,tt0045152,7.0
...,...,...,...
25000090,162541,tt0382932,9.0
25000091,162541,tt0389790,5.0
25000092,162541,tt0952640,4.0
25000093,162541,tt0468569,8.0


### Reformatting IMDb Reviews
We need to rename our IMDb review columns to match before we can merge with the MovieLens reviews.

In [45]:
# read in our IMDb reviews
imdb_reviews = pd.read_csv('../Data/Large-Data/ieee_imdb_reviews.csv')
imdb_reviews.head()

,UserID,MovieID,Rating,ReviewDate
0,ur4592644,tt0120884,10,2005-01-16
1,ur3174947,tt0118688,3,2005-01-16
2,ur3780035,tt0387887,8,2005-01-16
3,ur4592628,tt0346491,1,2005-01-16
4,ur3174947,tt0094721,8,2005-01-16


In [46]:
# Drop unneeded columns
imdb_reviews.drop(columns='ReviewDate', inplace=True)

In [47]:
# We need to rename our columns to match the MovieLens reviews
imdb_reviews = imdb_reviews.rename({'UserID': 'user_id', 'MovieID': 'imdb_id', 'Rating': 'scaled_rating'}, axis='columns')
imdb_reviews.head()

,user_id,imdb_id,scaled_rating
0,ur4592644,tt0120884,10
1,ur3174947,tt0118688,3
2,ur3780035,tt0387887,8
3,ur4592628,tt0346491,1
4,ur3174947,tt0094721,8


### Combining into one DataFrame
We will combine these reviews into one dataframe to have a unified set of reviews. We need to rename our columns to match and then we can concatonate vertically.

In [72]:
merged_reviews = pd.concat([ml_ratings_for_merge, imdb_reviews], axis=0)
merged_reviews

,user_id,imdb_id,scaled_rating
0,1,tt0110912,10.0
1,1,tt0111495,7.0
2,1,tt0108394,10.0
3,1,tt0114787,10.0
4,1,tt0045152,7.0
...,...,...,...
4669815,ur0581842,tt0107977,6.0
4669816,ur3174947,tt0103776,8.0
4669817,ur4592639,tt0107423,9.0
4669818,ur4581944,tt0102614,8.0


In [73]:
# Let's double check that the shape of our merged reviews is correct
print(ml_ratings_for_merge.shape)
print(imdb_reviews.shape)
print(merged_reviews.shape)

(25000095, 3)
(4669820, 3)
(29669915, 3)


### Adding in Title from IMDb Title Data
We will need the titles to show in our recommender system. We will merge them over from our IMDb Title data set

In [12]:
imdb_titles = pd.read_csv('../Data/Large-Data/imdb_titles.csv')
imdb_titles.head()

,titleId,title
0,tt0000001,Carmencita
1,tt0000002,Le clown et ses chiens
2,tt0000003,Pauvre Pierrot
3,tt0000004,Un bon bock
4,tt0000005,Blacksmith Scene


In [13]:
imdb_titles = imdb_titles.rename({'titleId': 'imdb_id'}, axis='columns')
imdb_titles.head()

,imdb_id,title
0,tt0000001,Carmencita
1,tt0000002,Le clown et ses chiens
2,tt0000003,Pauvre Pierrot
3,tt0000004,Un bon bock
4,tt0000005,Blacksmith Scene


In [76]:
merged_reviews = pd.merge(merged_reviews, imdb_titles, on= 'imdb_id', how='left')
merged_reviews

,user_id,imdb_id,scaled_rating,title
0,1,tt0110912,10.0,Pulp Fiction
1,1,tt0111495,7.0,Trois couleurs: Rouge
2,1,tt0108394,10.0,Trois couleurs: Bleu
3,1,tt0114787,10.0,Underground
4,1,tt0045152,7.0,Singin' in the Rain
...,...,...,...,...
29669910,ur0581842,tt0107977,6.0,Robin Hood: Men in Tights
29669911,ur3174947,tt0103776,8.0,Batman Returns
29669912,ur4592639,tt0107423,9.0,Lipstick on Your Collar
29669913,ur4581944,tt0102614,8.0,Out for Justice


In [77]:
merged_reviews.isna().sum()

user_id               0
imdb_id               0
scaled_rating         0
title            343723
dtype: int64

In [78]:
rows_with_null_title = merged_reviews[merged_reviews['title'].isnull()]

rows_with_null_title

,user_id,imdb_id,scaled_rating,title
5921,43,tt0118114,7.0,NaN
7427,59,tt0118114,8.0,NaN
7732,59,tt0099851,6.0,NaN
7855,61,tt0118114,10.0,NaN
11770,84,tt0424319,6.0,NaN
...,...,...,...,...
29668929,ur0562732,tt0354351,8.0,NaN
29669380,ur3553037,tt0093189,5.0,NaN
29669667,ur1637337,tt0092396,8.0,NaN
29669717,ur2571218,tt0385918,7.0,NaN


Just from quickly looking up these IMDb IDs on IMDb, some of these appear to be TV shows, other are not listed. I think it is safe to remove these since they only account for about 1% of our reviews.

In [79]:
# Let's drop the columns without titles
merged_reviews = merged_reviews.dropna()
merged_reviews.isna().sum()

user_id          0
imdb_id          0
scaled_rating    0
title            0
dtype: int64

In [51]:
# Let's save this to use for our recommender system
merged_reviews.to_csv('../Data/Large-Data/merged_reviews.csv', index = False)

## DataFrame Filtered by number of reviews

In [21]:
merged_reviews_filter = pd.read_csv('../Data/Large-Data/merged_reviews.csv')
merged_reviews_filter.head()

/var/folders/jc/vvd13n2d3m33dzsrsg9ydhb00000gn/T/ipykernel_1650/2949942281.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_reviews_filter = pd.read_csv('../Data/Large-Data/merged_reviews.csv')


,user_id,imdb_id,scaled_rating,title
0,1,tt0110912,10.0,Pulp Fiction
1,1,tt0111495,7.0,Trois couleurs: Rouge
2,1,tt0108394,10.0,Trois couleurs: Bleu
3,1,tt0114787,10.0,Underground
4,1,tt0045152,7.0,Singin' in the Rain


In [22]:
print(merged_reviews_filter.shape)

(29326192, 4)


In [23]:
merged_reviews_filter['imdb_id'].nunique()

244578

In [27]:
# Let's filter for movies that have more than 1 review
more_than_1 = merged_reviews_filter.groupby('imdb_id').filter(lambda x: len(x)>1)


29242970

In [29]:
# Let's filter for movies that have more than 5 reviews
more_than_5 = merged_reviews_filter.groupby('imdb_id').filter(lambda x: len(x)>5)
len(more_than_5['imdb_id'])

29011191

In [31]:
# Let's filter for movies that have more than 10 reviews
more_than_10 = merged_reviews_filter.groupby('imdb_id').filter(lambda x: len(x)>10)


28805995

In [32]:
# Let's filter for movies that have more than 20 reviews
more_than_20 = merged_reviews_filter.groupby('imdb_id').filter(lambda x: len(x)>20)


28511386

In [34]:
# Let's filter for movies that have more than 100 reviews
more_than_100 = merged_reviews_filter.groupby('imdb_id').filter(lambda x: len(x)>100)

In [35]:
# Let's check how much data we have for movies for each filter
print(f"More than 1: {len(more_than_1['imdb_id'])} reviews for {more_than_1['imdb_id'].nunique()} movies from {more_than_1['user_id'].nunique()} users")
print(f"More than 5: {len(more_than_5['imdb_id'])} reviews for {more_than_5['imdb_id'].nunique()} movies from {more_than_5['user_id'].nunique()} users")
print(f"More than 10: {len(more_than_10['imdb_id'])} reviews for {more_than_10['imdb_id'].nunique()} movies from {more_than_10['user_id'].nunique()} users")
print(f"More than 20: {len(more_than_20['imdb_id'])} reviews for {more_than_20['imdb_id'].nunique()} movies from {more_than_20['user_id'].nunique()} users")
print(f"More than 100: {len(more_than_100['imdb_id'])} reviews for {more_than_100['imdb_id'].nunique()} movies from {more_than_100['user_id'].nunique()} users")

More than 1: 29242970 reviews for 161356 movies from 1583709 users
More than 5: 29011191 reviews for 84319 movies from 1530647 users
More than 10: 28805995 reviews for 57405 movies from 1485005 users
More than 20: 28511386 reviews for 37275 movies from 1419983 users
More than 100: 27522152 reviews for 14339 movies from 1229633 users


In [36]:
# Let's save this to use for our recommender system
more_than_100.to_csv('../Data/Large-Data/over_100_reviews.csv', index = False)

## Filtering by number of reviews by user

In [38]:
reviews_min_1_per_user = more_than_100.groupby('user_id').filter(lambda x: len(x)>1)

In [39]:
reviews_min_5_per_user = more_than_100.groupby('user_id').filter(lambda x: len(x)>5)

In [40]:
reviews_min_10_per_user = more_than_100.groupby('user_id').filter(lambda x: len(x)>10)

In [ ]:
reviews_min_15_per_user = more_than_100.groupby('user_id').filter(lambda x: len(x)>15)

In [41]:
# Let's check how much data we have for movies for each filter
print(f"More than 1: {len(reviews_min_1_per_user['imdb_id'])} reviews for {reviews_min_1_per_user['imdb_id'].nunique()} movies from {reviews_min_1_per_user['user_id'].nunique()} users")
print(f"More than 5: {len(reviews_min_5_per_user['imdb_id'])} reviews for {reviews_min_5_per_user['imdb_id'].nunique()} movies from {reviews_min_5_per_user['user_id'].nunique()} users")
print(f"More than 10: {len(reviews_min_10_per_user['imdb_id'])} reviews for {reviews_min_10_per_user['imdb_id'].nunique()} movies from {reviews_min_10_per_user['user_id'].nunique()} users")

More than 1: 26759773 reviews for 14337 movies from 467254 users
More than 5: 26108686 reviews for 14324 movies from 226046 users
More than 10: 25848969 reviews for 14311 movies from 191100 users


###  Creating dataframe for MovieLens reviews only
We are having issues creating a pivot table with all the reviews

In [14]:
ml_ratings_for_merge.head()

,user_id,imdb_id,scaled_rating
0,1,tt0110912,10.0
1,1,tt0111495,7.0
2,1,tt0108394,10.0
3,1,tt0114787,10.0
4,1,tt0045152,7.0


In [15]:
ml_ratings_for_merge = pd.merge(ml_ratings_for_merge, imdb_titles, on= 'imdb_id', how='left')
ml_ratings_for_merge

,user_id,imdb_id,scaled_rating,title
0,1,tt0110912,10.0,Pulp Fiction
1,1,tt0111495,7.0,Trois couleurs: Rouge
2,1,tt0108394,10.0,Trois couleurs: Bleu
3,1,tt0114787,10.0,Underground
4,1,tt0045152,7.0,Singin' in the Rain
...,...,...,...,...
25000090,162541,tt0382932,9.0,Ratatouille
25000091,162541,tt0389790,5.0,Bee Movie
25000092,162541,tt0952640,4.0,Alvin and the Chipmunks
25000093,162541,tt0468569,8.0,The Dark Knight


In [16]:
ml_ratings_for_merge.isna().sum()

user_id              0
imdb_id              0
scaled_rating        0
title            30230
dtype: int64

In [18]:
# Let's drop the columns without titles
ml_ratings_for_merge = ml_ratings_for_merge.dropna()
ml_ratings_for_merge.isna().sum()

user_id          0
imdb_id          0
scaled_rating    0
title            0
dtype: int64

In [20]:
# Let's save this to use for our recommender system
ml_ratings_for_merge.to_csv('../Data/Large-Data/ml_reviews.csv', index = False)

# Next thing